In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
from torch.utils import data
import numpy as np
from PIL import Image
from torchvision import transforms
import os
import pyprind
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils import model_zoo
import dataloader
import scikitplot as skplt 

In [2]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=5, start_in_channels=64):
        super(ResNet, self).__init__()
        
        self.current_in_channels = start_in_channels
        
        self.first = nn.Sequential(
            nn.Conv2d(
                3, self.current_in_channels,
                kernel_size=7, stride=2, padding=3, bias=False
            ),
            nn.BatchNorm2d(self.current_in_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        
        self.layers = layers
        channels = self.current_in_channels
        for i, l in enumerate(layers):
            setattr(self, 'layer'+str(i+1), 
                    self._make_layer(block, channels, l, stride=(2 if i!=0 else 1) ))
            channels*=2
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.current_in_channels, num_classes)
            
    def _make_layer(self, block, in_channels, blocks, stride=1):
        downsample=None
        # 首先判斷 stride 是否為1，輸入通道和輸出通道是否相等。不相等則使用 1 X 1 的卷積改變大小和通道
        #作為 downsample
        # 在 Resnet 中，每層 layer 傳入的 stride =2
        if stride != 1 or self.current_in_channels != in_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.current_in_channels, in_channels * block.expansion,
                    kernel_size = 1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(in_channels * block.expansion)
            )
        
        layers = []
        # 新增第一個 basic block，把 downsample 傳給 BasicBlock 作為降取樣的層。
        layers.append(block(self.current_in_channels, in_channels, stride=stride, downsample=downsample))
        
        # 修改輸出的通道數
        self.current_in_channels = in_channels * block.expansion
        
        # 繼續新增這個 layer 裡接下來的 BasicBlock
        for i in range(1, blocks):
            layers.append(block(self.current_in_channels, in_channels))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.first(x)
        for i in range(len(self.layers)):
            x = getattr(self, 'layer'+str(i+1))(x)
        x = self.avgpool(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

In [3]:
# 兩個 3*3 前後的維度相同
class BasicBlock(nn.Module):
    '''
    x = (in, H, W) -> conv2d -> (out, H, W) -> conv2d -> (out, H, W) + x
    '''
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride=1, kernel_size=3, downsample=None):
        super(BasicBlock, self).__init__()
        padding = int(kernel_size/2)
        self.activation = nn.ReLU(inplace=True)
        self.block = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, 
                kernel_size=kernel_size, padding=padding, stride=stride, bias=False
            ),
            nn.BatchNorm2d(out_channels),
            self.activation,
            
            nn.Conv2d(
                out_channels, out_channels, 
                kernel_size=kernel_size, padding=padding, bias=False
            ),
            nn.BatchNorm2d(out_channels),
        )
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        #x 賦值給 residual，用於後面的 shortcut 連線
        out = self.block(x)
        
        if self.downsample is not None:
            #遇到降維或升維時要保證能夠相加
            residual = self.downsample(x)
        
        out += residual
        out = self.activation(out)
        
        return out


In [4]:
# Bottleneck：三個卷基層分别是1x1，3x3，1x1，分别用来降低维度，卷積處理，提高维度
# 目的是 -> 减少參數數量，Bottleneck相比较BasicBlock在参数的数目上較少但經度差不多

class BottleneckBlock(nn.Module):
    '''
    x = (in, H, W) -> conv2d(1x1) -> conv2d -> (out, H, W) -> conv2d(1x1) -> (out*4, H, W) + x 
    '''
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride=1, kernel_size=3, downsample=None):
        super(BottleneckBlock, self).__init__()
        padding = int(kernel_size/2)
        self.activation = nn.ReLU(inplace=True)
        self.block = nn.Sequential(
            # 1x1 的卷積是為了降維，減少通道數
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            self.activation,
            # 3x3 的卷積是為了改變圖片大小，不改變通道數
            nn.Conv2d(
                out_channels, out_channels,
                kernel_size=kernel_size, stride=stride, padding=padding, bias=False
            ),
            nn.BatchNorm2d(out_channels),
            self.activation,
            
             # 1x1 的卷積是為了升維，增加通道數，增加到 planes * 4
            nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * self.expansion),
        )
        self.downsample = downsample
    
    def forward(self, x):
        residual = x
        out = self.block(x)
        
        #若上一個Residual Block的輸出維度和當前維度不同，則對這個x進行downsample，若維度依樣則直接鄉間(out += residual)
        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = self.activation(out)
        
        return out

In [5]:
def resnet18(pretrained=False, **kwargs): 
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model
def resnet50(pretrained=False, **kwargs):
    model = ResNet(BottleneckBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
}

In [6]:
def train(model, data_loader, learning_rate, epoch):
    
    model.train()
    Loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)
    

    for batch_idx, (img, label) in enumerate(data_loader):

        data, target = img.to(device), label.to(device)
    
        optimizer.zero_grad()
        output = model(data)
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()

    if (epoch%1 == 0):    
        print("-------------------------------------------------")
        print("Train Epoch : ", epoch+1, "    Loss : ", loss.item())
        print("-------------------------------------------------")
        
    return model

def test(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    Loss = torch.nn.CrossEntropyLoss()
    
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += Loss(output, target).item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(data_loader) # loss function already averages over batch size
    
    print('\nTest set:  Accuracy: {}/{} ({}%)\n'.format(correct, len(data_loader.dataset), 
                                                        100. * correct / len(data_loader.dataset)))
    acc = 10000000*correct / len(data_loader.dataset)
    
    return acc

def get_predict(model, data_loader):
    model.eval()
    y_pred = []
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        output = output.data.max(1)[1]
        output = output.cpu()
        output = output.numpy()
        y_pred = np.append(y_pred, output)

    return y_pred

### Load Data

In [7]:
train_dataset = dataloader.RetinopathyLoader('./data', 'train')
test_dataset  = dataloader.RetinopathyLoader('./data', 'test')

augmentation = [
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
]
print("> dataset with augmentation with{}".format(augmentation))

train_dataset_with_augementation = dataloader.RetinopathyLoader('./data', 'train', augmentation=augmentation)

> Found 28100 images...
> Found 7026 images...
> dataset with augmentation with[RandomHorizontalFlip(p=0.5), RandomVerticalFlip(p=0.5)]
> Found 28100 images...


In [8]:
"""
    PARAMETER：
    For ResNet18
"""

Batch_size= 24
Learning_rate = 0.001 
device = torch.device("cuda:0" )

In [9]:
train_loader = DataLoader(train_dataset, batch_size=Batch_size)
train_aug_loader = DataLoader(train_dataset_with_augementation, batch_size=Batch_size)
test_loader = DataLoader(test_dataset, batch_size=Batch_size)

In [10]:
y_true = []
for batch_idx, (data, target) in enumerate(test_loader):     
    y_true = np.append(y_true, target)

### For ResNet18 without pretrained

In [11]:
ResNet18 = resnet18(pretrained=False)
ResNet18.to(device)

ResNet(
  (first): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (activation): ReLU(inplace=True)
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (activation): ReLU(inplace=True)
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [ ]:
Epochs = 10
acc_ResNet18 = 0
acc_ResNet18_test  = np.zeros(Epochs)
acc_ResNet18_train = np.zeros(Epochs)

for epoch in range(0, Epochs):
    print('Training epoch no.', epoch+1)
    ResNet18 = train(ResNet18, train_aug_loader, Learning_rate, epoch)
    acc_ResNet18_train[epoch] = test(ResNet18, train_loader)
    acc_ResNet18_test[epoch]  = test(ResNet18, test_loader)

    if (acc_ResNet18_test[epoch] > acc_ResNet18):
        torch.save({'state_dict': ResNet18.state_dict()}, 'model_ResNet18_w0_pre_epoch10_batch24.pth.tar')
        
        print("> New higher accuracy !!! ", acc_ResNet18_test[epoch]/100000)
        acc_ResNet18 = acc_ResNet18_test[epoch]
    print ('------------------------------------------------------------------------')

# torch.save({'state_dict': ResNet18.state_dict()}, 'model_ResNet18_w0_pre_epoch10_batch24_final.pth.tar')

Training epoch no. 1


In [ ]:
checkpoint = torch.load('model_ResNet18_w0_pre_epoch10_batch24.pth.tar')
ResNet18.load_state_dict(checkpoint['state_dict'])

acc_ResNet18 = test(ResNet18, test_loader)
acc_ResNet18_float = acc_ResNet18.item() / 100000


print ('------------------------------------------------------------------------')
print ('Highest accuracy for ResNet18 without pretrained : {} %'.format(acc_ResNet18_float))
print ('------------------------------------------------------------------------')

In [ ]:
# Plot the confusion matrix
y_ResNet18_without = get_predict(ResNet18, test_loader)

skplt.metrics.plot_confusion_matrix(y_true, y_ResNet18_without, normalize=True)

In [ ]:
# store acc_ResNet18_test, acc_ResNet18_train to csv
acc_ResNet18_train_float = np.zeros(len(acc_ResNet18_train))
acc_ResNet18_test_float = np.zeros(len(acc_ResNet18_test))
for i in range(len(acc_ResNet18_test)):
    acc_ResNet18_test_float[i] = acc_ResNet18_test[i]/100000
    acc_ResNet18_train_float[i] = acc_ResNet18_train[i]/100000
pd.DataFrame(acc_ResNet18_test_float).to_csv("result/ResNet18_test_epoch10_batch24.csv")
pd.DataFrame(acc_ResNet18_train_float).to_csv("result/ResNet18_train_epoch10_batch24.csv")

In [ ]:
del ResNet18
torch.cuda.empty_cache() 

### ResNet18 with pretrained model

In [ ]:
ResNet18_pretrained = models.resnet18(pretrained=True)
fc_features = ResNet18_pretrained.fc.in_features
ResNet18_pretrained.fc = nn.Linear(fc_features, 5)
ResNet18_pretrained.to(device)

In [ ]:
Epochs = 10
acc_ResNet18_pretrained = 0
acc_ResNet18_pretrained_test  = np.zeros(Epochs)
acc_ResNet18_pretrained_train = np.zeros(Epochs)

for epoch in range(0, Epochs):
    print('Training epoch no.', epoch+1)
    ResNet18_pretrained = train(ResNet18_pretrained, train_loader, Learning_rate, epoch)
    acc_ResNet18_pretrained_test[epoch] = test(ResNet18_pretrained, test_loader)
    
    if (acc_ResNet18_pretrained_test[epoch] > acc_ResNet18_pretrained):
        torch.save({'state_dict': ResNet18_pretrained.state_dict()}, 'model_ResNet18_pretrained_epoch10_batch24.pth.tar')
        print("New higher accuracy!!",acc_ResNet18_pretrained_test[epoch])
        acc_ResNet18_pretrained = acc_ResNet18_pretrained_test[epoch]
    print ('------------------------------------------------------------------------')
torch.save({'state_dict': ResNet18_pretrained.state_dict()}, 'model_ResNet18_pretrained_epoch10_batch24_final.pth.tar')

In [ ]:
checkpoint = torch.load('model_ResNet18_pretrained_epoch10_batch24.pth.tar')
ResNet18_pretrained.load_state_dict(checkpoint['state_dict'])

acc_ResNet18_pretrained = test(ResNet18_pretrained, test_loader)
acc_ResNet18_pretrained_float = acc_ResNet18_pretrained.item() / 100000


print ('------------------------------------------------------------------------')
print ('Highest accuracy for ResNet18 with pretrained : {} %'.format(acc_ResNet18_pretrained_float))
print ('------------------------------------------------------------------------')


In [ ]:
y_ResNet18_with = get_predict(ResNet18_pretrained, test_loader)

skplt.metrics.plot_confusion_matrix(y_true, y_ResNet18_with, normalize=True)

In [ ]:
# store acc_ResNet18_pretrained_test to csv
acc_ResNet18_pretrained_test_float = np.zeros(len(acc_ResNet18_pretrained_test))
acc_ResNet18_pretrained_train_float = np.zeros(len(acc_ResNet18_pretrained_train))

for i in range(len(acc_ResNet18_pretrained_test)):
    acc_ResNet18_pretrained_test_float[i]  = acc_ResNet18_pretrained_test[i]/100000
    acc_ResNet18_pretrained_train_float[i] = acc_ResNet18_pretrained_train[i]/100000
    
pd.DataFrame(acc_ResNet18_pretrained_test_float).to_csv("result/ResNet18_pretrained_test_epoch10_batch24.csv")
pd.DataFrame(acc_ResNet18_pretrained_train_float).to_csv("result/ResNet18_pretrained_train_epoch10_batch24.csv")

In [ ]:
del ResNet18_pretrained
torch.cuda.empty_cache() 

In [ ]:
"""
    PARAMETER
    For ResNet50
"""
Batch_size= 10
Learning_rate = 0.001 
device = torch.device("cuda:1" )

### For ResNet50 wothout pretrained

In [ ]:
ResNet50 = resnet50(pretrained=False)
ResNet50.to(device)

In [ ]:
Epochs = 5
acc_ResNet50 = 0
acc_ResNet50_test = np.zeros(Epochs)
acc_ResNet50_train = np.zeros(Epochs)

for epoch in range(0, Epochs):
    print('Training epoch no.', epoch+1)
    ResNet50 = train(ResNet50, train_aug_loader, Learning_rate, epoch)
    acc_ResNet50_train[epoch] = test(ResNet50, train_loader)
    acc_ResNet50_test[epoch]  = test(ResNet50, test_loader)
    
    if (acc_ResNet50_test[epoch] > acc_ResNet50):
        torch.save({'state_dict': ResNet50.state_dict()}, 'model_ResNet50_w0_pre_epoch5_batch10.pth.tar')
        
        print("New higher accuracy!!",acc_ResNet50_test[epoch]/100000)
        acc_ResNet50 = acc_ResNet50_test[epoch]
    print ('------------------------------------------------------------------------')
    
# torch.save({'state_dict': ResNet50.state_dict()}, 'model_ResNet50_w0_pre_epoch5_batch10_final.pth.tar')

In [ ]:
checkpoint = torch.load('model_ResNet50_w0_pre_epoch5_batch10.pth.tar')
ResNet50.load_state_dict(checkpoint['state_dict'])

acc_ResNet50 = test(ResNet50, test_loader)
acc_ResNet50_float = acc_ResNet50.item() / 100000


print ('------------------------------------------------------------------------')
print ('Highest accuracy for ResNet50 without pretrained : {} %'.format(acc_ResNet50_float))
print ('------------------------------------------------------------------------')

In [ ]:
y_ResNet50_without = get_predict(ResNet50, test_loader)

skplt.metrics.plot_confusion_matrix(y_true, y_ResNet50_without, normalize=True)

In [ ]:
# store acc_ResNet50_test, acc_ResNet50_trainto csv
acc_ResNet50_test_float = np.zeros(len(acc_ResNet50_test))
acc_ResNet50_train_float = np.zeros(len(acc_ResNet50_train))

for i in range(len(acc_ResNet50_test)):
    acc_ResNet50_test_float[i] = acc_ResNet50_test[i]/100000
    acc_ResNet50_train_float[i] = acc_ResNet50_train[i]/100000
    
pd.DataFrame(acc_ResNet50_test_float).to_csv("result/ResNet50_test_epoch5_batch10.csv")
pd.DataFrame(acc_ResNet50_train_float).to_csv("result/ResNet50_train_epoch5_batch10.csv")

In [ ]:
del ResNet50
torch.cuda.empty_cache() 

### For ResNet50 with pretrained

In [ ]:
ResNet50_pretrained = models.resnet50(pretrained=True)
fc_features = ResNet50_pretrained.fc.in_features
ResNet50_pretrained.fc = nn.Linear(fc_features, 5)
ResNet50_pretrained.to(device)

In [ ]:
Epochs = 5
acc_ResNet50_pretrained = 0
acc_ResNet50_pretrained_test  = np.zeros(Epochs)
acc_ResNet50_pretrained_train = np.zeros(Epochs)

for epoch in range(0, Epochs):
    print('Training epoch no.', epoch+1)
    ResNet50_pretrained = train(ResNet50_pretrained, train_aug_loader, Learning_rate, epoch)
    acc_ResNet50_pretrained_train[epoch] = test(ResNet50_pretrained, train_loader)
    acc_ResNet50_pretrained_test[epoch]  = test(ResNet50_pretrained, test_loader)
    
    if (acc_ResNet50_pretrained_test[epoch] > acc_ResNet50_pretrained):
        torch.save({'state_dict': ResNet50_pretrained.state_dict()}, 'model_ResNet50_pretrained_epoch5_batch10.pth.tar')
        print("New higher accuracy!!",acc_ResNet50_pretrained_test[epoch]/100000)
        acc_ResNet50_pretrained = acc_ResNet50_pretrained_test[epoch]
    print ('------------------------------------------------------------------------')
# torch.save({'state_dict': ResNet50_pretrained.state_dict()}, 'model_ResNet50_pretrained_epoch5_batch10_final.pth.tar')

In [ ]:
checkpoint = torch.load('model_ResNet50_pretrained_epoch5_batch10.pth.tar')
ResNet50_pretrained.load_state_dict(checkpoint['state_dict'])

acc_ResNet50_pretrained = test(ResNet50_pretrained, test_loader)
acc_ResNet50_pretrained_float = acc_ResNet50_pretrained.item() / 100000

print ('------------------------------------------------------------------------')
print ('Highest accuracy for ResNet50 with pretrained : {} %'.format(acc_ResNet50_pretrained_float))
print ('------------------------------------------------------------------------')

In [ ]:
y_ResNet50_pre = get_predict(ResNet50_pretrained, test_loader)

skplt.metrics.plot_confusion_matrix(y_true, y_ResNet50_pre, normalize=True)

In [ ]:
# store acc_ResNet50_pretrained_test, acc_ResNet50_pretrained_train to csv
acc_ResNet50_pretrained_train_float = np.zeros(len(acc_ResNet50_pretrained_train))
acc_ResNet50_pretrained_test_float = np.zeros(len(acc_ResNet50_pretrained_test))

for i in range(len(acc_ResNet50_pretrained_test)):
    acc_ResNet50_pretrained_test_float[i]  = acc_ResNet50_pretrained_test[i]/100000
    acc_ResNet50_pretrained_train_float[i] = acc_ResNet50_pretrained_train[i]/100000
    
pd.DataFrame(acc_ResNet50_pretrained_test_float).to_csv("result/ResNet50_pretrained_test_epoch5_batch10.csv")
pd.DataFrame(acc_ResNet50_pretrained_train_float).to_csv("result/ResNet50_pretrained_train_epoch5_batch10.csv")

In [ ]:
del ResNet50_pretrained
torch.cuda.empty_cache() 

### Plot Result

In [ ]:
def to_float(df):
    df_np = df.to_numpy()
    df_np_col = df_np[:,1]
    del df_np
    for i in range(len(df_np_col)):
        df_np_col[i]  = df_np_col[i]/100
    return df_np_col

In [ ]:
ResNet18_w0_pre_test_epoch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet18_test_epoch10_batch20.csv') 
ResNet18_w0_pre_train_epoch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet18_train_epoch10_batch20.csv')
ResNet18_pretrained_test_epoch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet18_pretrained_test_epoch10_batch20.csv')
ResNet18_pretrained_train_epoch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet18_pretrained_train_epoch10_batch20.csv')

In [ ]:
ResNet18_w0_pre_test_epoch10 = to_float(ResNet18_w0_pre_test_epoch10)
ResNet18_w0_pre_train_epoch10 = to_float(ResNet18_w0_pre_train_epoch10)
ResNet18_pretrained_test_epoch10 = to_float(ResNet18_pretrained_test_epoch10)
ResNet18_pretrained_train_epoch10 = to_float(ResNet18_pretrained_train_epoch10)

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(ResNet18_w0_pre_test_epoch10)
plt.plot(ResNet18_w0_pre_train_epoch10)
plt.plot(ResNet18_pretrained_test_epoch10)
plt.plot(ResNet18_pretrained_train_epoch10)

plt.hlines(0.82, 0, 10, colors = "gray", linestyles = "dashed")
plt.hlines(0.8, 0, 10, colors = "gray", linestyles = "dashed")
plt.hlines(0.75, 0, 10, colors = "gray", linestyles = "dashed")


plt.title('ResNet18 comparison')
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.legend(['ResNet18 without pretrain_train data', 'ResNet18 without pretrain_test data',
            'ResNet18 with pretrain_train data', 'ResNet18 with pretrain_test data'], loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)

plt.show()

In [ ]:
ResNet50_test_epoch5_batch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet50_test_epoch5_batch10.csv') 
ResNet50_train_epoch5_batch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet50_train_epoch5_batch10.csv')
ResNet50_pretrained_test_epoch5_batch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet50_pretrained_test_epoch5_batch10.csv')
ResNet50_pretrained_train_epoch5_batch10 = pd.read_csv('/home/shaowen0213/IOC_DLP_110/Lab04/result/ResNet50_pretrained_train_epoch5_batch10.csv')

In [ ]:
ResNet50_test_epoch5_batch10 = to_float(ResNet50_test_epoch5_batch10)
ResNet50_train_epoch5_batch10 = to_float(ResNet50_train_epoch5_batch10)
ResNet50_pretrained_test_epoch5_batch10 = to_float(ResNet50_pretrained_test_epoch5_batch10)
ResNet50_pretrained_train_epoch5_batch10 = to_float(ResNet50_pretrained_train_epoch5_batch10)

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(ResNet50_test_epoch5_batch10)
plt.plot(ResNet50_train_epoch5_batch10)
plt.plot(ResNet50_pretrained_test_epoch5_batch10)
plt.plot(ResNet50_pretrained_train_epoch5_batch10)

plt.hlines(0.82, 0, 5, colors = "gray", linestyles = "dashed")
plt.hlines(0.8, 0, 5, colors = "gray", linestyles = "dashed")
plt.hlines(0.75, 0, 5, colors = "gray", linestyles = "dashed")


plt.title('ResNet50 comparison')
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.legend(['ResNet50 without pretrain_train data', 'ResNet50 without pretrain_test data',
            'ResNet50 with pretrain_train data', 'ResNet50 with pretrain_test data'], 
           loc=2, bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)

plt.show()

In [ ]:
from texttable import Texttable
print("     ResNet18, ResNet50 Highest result     ")
print("> ResNet18 with epoch:10, batch_size:20 ")
print("> ResNet50 with epoch:05, batch_size:10 ")
tb=Texttable()
tb.set_cols_align(['l', 'c', 'c'])
tb.set_cols_dtype(['t', 'f', 'f'])
tb.add_rows([['Net', 'without pretrained', 'with pretrained'],
            ["ResNet18", np.max(ResNet18_w0_pre_test_epoch10), np.max(ResNet18_pretrained_test_epoch10)],
            ["ResNet50", np.max(ResNet50_test_epoch5_batch10),  np.max(ResNet50_pretrained_train_epoch5_batch10)],
            ])
print(tb.draw())


# tb.add_rows(df.values, header=False)
# # tb.add_rows(['Net', 'ReLU', 'Leaky ReLU', 'ELU'])
# print(tb.draw())